# Kernel Implementation API:

This workbook shows how we use Numba as part of the DEC++ pipeline to write Python applications that will reap the most benefits from various features of the DECADES platform. Performance gains can be achieved through compilation for optimal memory access and optimal processor mapping, and the DECADES Numba Library allows for faster implementations of your Python algorithms. 

To learn more about how to execute your kernels using the DEC++ pipeline please read the [Decades Numba Pipeline](Decades_Numba_Pipeline.ipynb) document.

We will first give a short introduction on how to use Numba with Python. You can find more detailed information about how to use Numba [here](https://numba.pydata.org). We will then describe the DECADES Numba Library in detail to help you get familiarized with this toolbox.

Finally, we will have a short section for tips and tricks for adapting your code to Numba.

Let's first load a few useful libraries. Note that unless you launch the Jupyter notebook from a computer with the full environment, you cannot execute all of the cells below successfully.

In [1]:
import numpy as np
import numba
from numba import jit, njit, float32, int32, int64, jitclass
from time import time
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

### 1. Numba:

Numba is a specialized compiler for Python apps for optimizing numerically heavy code. The feature of Numba that we are most interested in is its use of LLVM to compile Python code (read more about LLVM [here](https://llvm.org)). It therefore at some point during compilation generates an LLVM IR file. This is useful for us because DEC++ compiler is also based on the LLVM architecture and we can insert DEC++ compilation into the Numba compilation path to optimize Python code for specialized hardware.

Given the limited set of variable types and functions Numba can handle at the moment, not every Python function can be Numba-compiled. We will use Numba only for functions that rely on numerical computations and that have heavy cpu usage. You can find out more about Numba [here](http://numba.pydata.org/numba-doc/latest/user/5minguide.html)

The Python interface of Numba allows wrapping of a given Python function such that it will first be fed into the Numba compiler before execution. Wrapping a function involves placing a `@jit` annotation above it.

For example, let's start with a matrix addition function:

In [3]:
def matrixSum(A, B):
    m, n = A.shape
    X = np.zeros_like(A)

    for i in range(m):
        for j in range(n):
            X[i, j] = A[i, j] + B[i, j]

    return X

Now let's make this function Numba-compilable:

In [2]:
@jit(nopython=True)
def matrixSum_numba(A, B):
    m, n = A.shape
    X = np.zeros_like(A)

    for i in range(m):
        for j in range(n):
            X[i, j] = A[i, j] + B[i, j]

    return X

In [4]:
X1 = np.random.uniform(low=0, high=1, size=1000000).reshape(1000, 1000)
X2 = np.random.uniform(low=0, high=1, size=1000000).reshape(1000, 1000)

t = time()
Z = matrixSum(X1, X2)
print('compute time without numba', time() - t)
t = time()
Z = matrixSum_numba(X1, X2)
print('compute time with numba', time() - t)

compute time without numba 0.3348517417907715
compute time with numba 0.19780993461608887


Notice that the only difference in writing the code with and without the Numba compiler is the presence of the decorator *@jit(nopython=True)*. It's that simple! The *@jit()* decorator allows the function to be wrapped for the Numba compiler.

#### 1.1 <span style="background-color: #F8E0F7"><font color="black"> nopython = True</span>

Numba can compile in two modes: object mode and the nopython mode. When the 
<span style="background-color: #F8E0F7"><font color="black"> nopython=True</span> is set, it will run in  *nopython* mode and the code will be compiled entirely without the involvement of the Python Interpreter. <br> <br>This is an important setting for us. When <span style="background-color: #F8E0F7"><font color="black"> nopython=True</span> is not set, in the event that Numba fails to compile, *@jit()* will use the Python interpreter without giving an error. For our applications we always want to make sure that Numba compiles the function entirely.
<br> <br> As a quick note, *@jit(noython=True)* can also be written shortly as *@njit()*.

#### 1.2 <span style="background-color: #F8E0F7"><font color="black"> @jit </span> vs <span style="background-color: #F8E0F7"><font color="black">@cc</span>

Another mode of compilation for Numba is the distinction between just-in-time (JIT) and ahead-of-time (AOT) compiler and these have different decorators and synthaxes. 

Here is an example of AOT mode:

In [5]:
from numba.pycc import CC

cc = CC('matrixSum')

@cc.export('sumf', 'f8[:,::1](f8[:,::1], f8[:,::1])')
@jit(nopython=True)
def matrixSum_AOT(A,B):
    m,n = A.shape
    X = np.zeros_like(A)

    for i in range(m):
        for j in range(n):
            X[i,j] = A[i,j] + B[i,j]

    return X

cc.compile()

While in JIT mode, Numba will compile the function on the fly during execution. AOT mode on the other hand will generate an binary executable that can be imported later. In the AOT mode, the compiler must know all possible versions of input and output variable types for compilation. In the case of JIT however, Numba can dynamically determine the variable type based on the input type provided. 

Although we would have liked to use the AOT, this functionality of Numba is not as well developed as the JIT mode. We will therefore be using the JIT, however we will provide all the input and output variable types just as it is done in the AOT mode. This will ensure that when we introduce the DEC++ compiler into the Numba pipeline it will run smoothly.

Now let's take a look at what that looks like:

In [ ]:
from numba import int32, njit                                                                                       
from numba.types import UniTuple                

@njit(UniTuple(int32, 2)(int32, int32), nogil=True)
def k_n_to_i_j(k, n):
    #function definition here
    #
    return i, j

The <span style="background-color: #F8E0F7"><font color="black"> nogil=True </span> setting is to make sure that Python's Global Interpreter Lock is released and Numba and DEC++ can now run the code on multiple threads.

#### 1.3 <span style="background-color: #F8E0F7"><font color="black"> @jitclass </span>

The last type of decorator we will talk about is jitclass() which is a Numba compiler for class type objects.
As usual all the variables belonging to a class must be specified in the decorator.

Here is an example of Numba compiled class, `spec`:

In [ ]:
from numba import jitclass,int32

spec = [('indptr', int32[:]), ('indices', int32[:])]

@jitclass(spec)
class SparseGraph:
    def __init__(self, graph_indptr, graph_indices):
        self.indptr = graph_indptr
        self.indices = graph_indices

### 2. DEC++ Pipeline:

Adding the DECADES pipeline is fairly simple. We simply need to import the DEC_Pipeline module and then use pipeline_class=DEC_pipeline in our decorator  settings. Let's now see the same code above with the DEC++ pipeline:

In [ ]:
import numpy as np
from numba import jitclass, int32, njit, float32
from numba.types import UniTuple
from DEC_Pipeline import DEC_Pipeline

@njit(UniTuple(int32, 2)(int32, int32), nogil=True, pipeline_class=DEC_Pipeline)
def k_n_to_i_j(k, n):
    #function definition here
    #
    return i, j

The above code will now be compiled through DEC++. You can learn more about specific kernel execution settings [here.](Decades_Numba_Pipeline.ipynb)


### 3. DEC_Numba_Lib :

To facilitate rapid development in Python using the DECADES pipeline, we developed a DECADES Numba Library focusing on the graph applications. We call this library DEC_Numba_Lib. You can import this library using <span style="background-color: #F8E0F7"><font color="black"> import DEC_Numba_Lib</span>. There are four different types of support we provide through this library:
 
#### 3.1 Load Data:
Currently , we provide two functions <span style="background-color: #F8E0F7"><font color="black"> LoadDecSparseGraph </span> and  <span style="background-color: #F8E0F7"><font color="black">LoadDecBipartiteGraph</span>  to load graph data into Numba compiled graph class objects, both of which take data directory path as input.  We plan on building on this library to further enable rapid development with Python for Numba DEC++ compilation.
<br>
<br>    
With LoadDecSparseGraph the edge and node_attribute data are loaded into a graph object (G) that has the edge data in CSR matrix format (G.indptr, G.indices, similar to SciPy csr matrix format) as well as node attributes (G.node_attr). LoadDecBipartiteGraph on the other hand loads the edge data from a bipartite graph to a triangular matrix.
<br>
<br>
Feel free to use these functions as well as the Numba-compiled DecSparseGraph and DecBipartiteGraph jitclasses they return as a basis to develop more complex graph data structures and Numba-compiled graph applications you might be developing.
<br>
<br>

#### 3.2 Data structures:
We have the following data structures in DEC_Numba_Lib:
Graph Data Classes:
 - **DecSparseGraph:** Graph object (see above) built as a Numba-compiled class with CSR matrix structure for adjacency matrix as well as node atributes.
 <br>
 <br>
 - **BiPartiteGraph:** Graph object built as a numba-compiled class specialized for bipartite graphs with a triangular matrix structure for edge data.

Here is an example using DecSparseGraph module of DECADES Numba Library:

In [ ]:
import numpy as np
from numba import njit
from numba.types import Tuple

from DEC_Pipeline import DEC_Pipeline
from DEC_Numba_Lib import DecSparseGraph, LoadDecSparseGraph, DecSparseGraphSpec

@njit(Tuple((int32, float32))(DecSparseGraphSpec(), int32[:], int32), nogil=True, pipeline_class=DEC_Pipeline)
def vertex_nomination_kernel_(G, seeds):
    # function definition here
    #
    return top_nominee, top_score

G = LoadDecSparseGraph(input_directory)
seeds = np.random.choice(num_nodes_connected, 5, replace=False)
top_nominee, top_score = vertex_nomination_kernel_(G, seeds)

Because Numba does not support SciPy library, we included Numba implementations of some of the most popular SciPy data structures and functions (decribed in the next section):
<br><br>
SciPy like Sparse Matrix Classes ([wikipedia](https://en.wikipedia.org/wiki/Sparse_matrix)):
<br>
<br>
 - **DecCSR:** Compressed Sparse Row format with row-wise stacked vector form matrix data. The class object has indptr, indices, data, shape and size instance variables. 
<br>
<br> 
 - **DecCSC:** Compressed Sparse Column format with column-wise stacked vector form matrix data. The class object has indptr, indices, data, shape and size instance variables. 
 <br>
<br>
 - **DecCOO:** Coordinate list format (row index, column index and data are stored in 3 different vectors). The class object has rows, cols, data, shape and size instance variables. 
 <br>
<br>

For all the data structures we provide special functions to retrieve class signatures to be used in the njit decorators of functions that would use these data as input (e.g. the spec for the DecSparseGraph class can be retrieved with DecSparseGraphSpec()). This removes the overhead of heavy variable typing prior to function calls and results in cleaner code.
<br>
<br>
You can load the graph data using LoadDecSparseGraph and build these different sparse matrices using instance variables of the resulting DecSparseGraph.
<br>
<br>
In the next section we describe the special functions in the DEC_Numba_Lib targeting these data structures such as conversion between different data structures and basic matrix operations.


In [ ]:
import numpy as np
from numba import njit
from numba.types import Tuple

from DEC_Pipeline import DEC_Pipeline
from DEC_Numba_Lib import DecSparseGraph, LoadDecSparseGraph, DecSparseGraphSpec

njit(Tuple((int32, float32))(DecSparseGraphSpec(), int32[:], int32), nogil=True, pipeline_class=DEC_Pipeline)
def vertex_nomination_kernel_(G, seeds):
    # function definition here
    #
    return top_nominee, top_score

G = LoadDecSparseGraph(input_directory)
seeds = np.random.choice(num_nodes_connected, 5, replace=False)
top_nominee, top_score = vertex_nomination_kernel_(G, seeds)

#### 3.3 Kernel Functions:
Matrix format conversion:
 - **coo_to_csr**
 - **csr_to_coo**
 - **csr_to_csc**
 - **csc_to_csr**
 
Matrix operations:
 - *Transpose:* **transpose_csc**, **transpose_csr**, **transpose_coo**
 <br>
 <br>
 - *Dot Product:* **dot** This function is capable of handling any combination of DecCSR, DecCSC, DecCOO and dense numpy matrices. Please note however, due to a Numba [bug](https://github.com/numba/numba/issues/3590) this function currently is on hold. You can however access the underlying specific implementations for different combinations of matrix data types directly inside the DEC_Numba_Lib module.
<br>
<br>
 - *Elemenwise Operations of sparse matrices:* **elementwise_sparse_sparse**, **elementwise_dense_sparse**, **elementwise_sparse_dense** (capable of handling elementwise multiplication of any combination of DecCSR, and dense numpy matrices)
<br>
<br> 
 - *Elementwise operations of numpy matrices:* Although numpy is supported, we also provide the numba compiled elementwise operations of numpy matrices in this library (**dec_mul_scalar**(scalar-matrix multiplication), **dec_add** (elementwise addition), **dec_subt**(elementwise subtraction), dec_div(elementwise division), **dec_dot**(dot product), **dec_int64_max**(find maximum))
 
 Other miscellaneous operations:
 - **dec_minimum_spanning_tree**: finds the minimum spanning tree given a DecCSR graph data structure using Kruskal's algorithm.
 <br>
 <br>
 - **eliminate_zeros**: eliminates explicit zeros from DecCSR Matrix.
 <br>
 <br>
 - **list_intersection**: find intersection of two lists, especially useful for common neighbor search.
 <br>
 <br>
 - Triangular matrix helper functions: **TriDenseGraph_k_to_i_j**, **TriDenseGraph_num_ele_i_rows**)
 
<br>
<br>
 
#### 3.4 Synchronization:
Finally, we also provide the **PyDECADES_barrier** function to synchronize all the threads in order to preserve memory access order in the [multi-threaded operations](Decades_Numba_Pipeline.ipynb). This can be achieved by inserting PyDECADES_barrier() in the main kernel Python app to the point in the program where all the threads must complete operations in order to proceed to the next set of instructions. 


<br> 

### Appendix 2 Tips and tricks with Numba:

 - **Most common error you will encounter:** By far the most common error you will come across is going to be due to variable type settings. You will get an error similar to "TypeError: No matching definition for argument type(s) array(float32, 1d, C), array(int64, 1d, C), array(int64, 1d, C)" in such cases. You need to make sure that Python and Numpy variables are cast into specific data types your Numba function expects. Numba does not give very good description of the issues in the error, so you might need to do some detective work to understand which variable or function it is complaining about.
<br><br>

 - **Python default integer type:** Numba type of a python generated default integer is int64. You may need to cast it to a different type if this is not what you wanted in the numba code. You can do `np.int32(x)` if you want to cast it to int32 for instance.
<br><br>


 - **Assigning values to output array:** When you construct a kernel to be launched by the `decades_kernel_launcher`, you will need to provide the return output as an initialized input. If this variable is an array, in order to assign values to the entire array you would normally do `returnarray = ...` However this will result in creation of a new object inside the numba function rather than changing its content. You therefore have to do `returnarray[:] = ...` . This is not an issue if you need to do this: `returnarray[i]=...`.
 <br><br>


 - **List of objects:** If you need to have a list of objects in your input arguments, the signature you need to use for that is `List(ObjectType)`. 
  <br><br>


 - **Appending to numpy arrays:** It is very costly to do appending or concatenation operations with numpy arrays. Therefore we recommend using fixed size arrays (with an initial max size) and keep track of the end of the array using an integer variable.
  <br><br>


 - **Index out of range:** Numba unfortunately will let you assign a value out or range of an array and result in `core dump`. If you see this issue, inpect your code to be sure you are not trying to access a location out of range of an array.
  <br><br>


 - **Consistent variable types:** For any operation inside a Numba compiled function, you need to have consistent variable types. to give an example in `np.zeros((a, b))`, `a` and `b` must have the same integer type.
 <br><br>


 - **Lists versus numpy arrays:** Although Numba supports lists, note that it does not support list of lists. It would be wise to use numpy arrays instead of lists. This way, you will be less prone to coming across errors and bugs related to whether or not Numba can handle the python  functions associated with lists. There may however be situations where using lists is much more convenient, in those cases make sure the types of the variables inside the list are compatible with operations you will do with them.
 <br><br>
 - **Variable types in Numba**:
Since you will need to declare the type of the input and output variables of your njit function, let's talk about different variable types you might have to deal with. 
As we said above, safest type of variables to use in Numba are Numpy variables where most of the methods associated with these variables are supported.

Commonly used variable types are as follows:<br>

<table>
      <tr><td><font size="3">int32</td><td> <font size="3">integer variable</td></tr>
      <tr><td><font size="3">int32[:] </td><td><font size="3">integer array</td></tr>
      <tr><td><font size="3">int32[:,:] </td><td><font size="3">2D integer array</td></tr>
      <tr><td><font size="3">int32[:,::1] </td><td><font size="3">2D integer array, packed row-wise</td></tr>
    <tr><td><font size="3">... </td><td>...</td></tr>
</table>

where int32 can be replaced by int8, int16, int64, float32 and float64. The output variables must be wrapped in a Numba Tuple if there are more than one. 

You can also use jitclasses as input and output variables and in that case you need to capture the class signature by adding .class_type.instance_type at the end of a class name and use this signature as variable type.

For more information on different variable types of Numba, please refer [here](https://numba.pydata.org/numba-doc/dev/reference/types.html)